In [50]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [4]:
dsf1 = tabula.read_pdf('Topcondoimage 06-06-2022.pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 06-13-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 06-20-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 06-27-2022.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 07-04-2022.pdf', pages='all',lattice=True)

In [5]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]

In [6]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]

In [7]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])

In [8]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]

In [9]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [10]:
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [11]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [13]:
x = df['Close Date'].str.startswith('6')
df = df[x]

In [14]:
len(df)

45

In [15]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,Jade Signature\r16901 Collins Ave 5305\rSunny ...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,Prime International Assoc Inc,Jade Signature 16901 Collins Ave 5305 Sunny Is...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach
1,Bayview\r5313 Fisher Island Dr 5313\rMiami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,Luxury Living Realty,Bayview 5313 Fisher Island Dr 5313 Miami Beach,Bayview,5313 Fisher Island Dr 5313 Miami Beach
5,Club Tower One\r789 Crandon Blvd PH-6\rKey Bis...,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,BHHS EWM Realty,Club Tower One 789 Crandon Blvd PH-6 Key Biscayne,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne
6,1 Hotel & Homes\r102 24th St 1520\rMiami Beach,10/1/2021,6/3/2022,189,"$5,100,000.00",1386,"$3,679.65",Jennifer Restrepo,EXP Realty LLC,Jennifer Restrepo,EXP Realty LLC,1 Hotel & Homes 102 24th St 1520 Miami Beach,,1 Hotel & Homes 102 24th St 1520 Miami Beach
8,Two Park Grove\r2821 S Bayshore Dr 10B\rMiami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,One Sotheby's Intl Realty,Two Park Grove 2821 S Bayshore Dr 10B Miami,Two Park Grove,2821 S Bayshore Dr 10B Miami
10,Eighty Seven Park\r8701 Collins Ave 1102/1106\...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,Douglas Elliman,Eighty Seven Park 8701 Collins Ave 1102/1106 M...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach
11,321 Ocean Condo\r321 Ocean Dr BH200\rMiami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,"Compass Florida, LLC",321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach
12,Turnberry Ocean Club\r18501 Collins Ave 4002\r...,3/25/2022,6/6/2022,41,"$5,500,000.00",2821,"$1,949.66",Carolina Arenas,Brown Harris Stevens,Richard Goihman,Douglas Elliman,Turnberry Ocean Club 18501 Collins Ave 4002 Su...,Turnberry Ocean Club,18501 Collins Ave 4002 Sunny Isles Beach
13,One Ocean\r1 Collins Ave 307\rMiami Beach,9/8/2021,6/7/2022,209,"$4,550,000.00",3041,"$1,496.22",Anna Sherrill,One Sotheby's Intl Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,One Ocean 1 Collins Ave 307 Miami Beach,One Ocean,1 Collins Ave 307 Miami Beach
14,Chateau Beach\r17475 Collins Ave 1001\rSunny I...,7/1/2021,6/6/2022,192,"$4,500,000.00",5210,"$1,063.83",Richard Goihman,Douglas Elliman,Darin Spencer Raymond PA,BHHS EWM Realty,Chateau Beach 17475 Collins Ave 1001 Sunny Isl...,Chateau Beach,17475 Collins Ave 1001 Sunny Isles Beach


In [16]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [17]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [18]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [19]:
df['loc']

0     (16901 Collins Ave #5305, Sunny Isles Beach, F...
1     (5313 Fisher Island Dr #5313, Miami, FL 33109,...
5     (789 Crandon Blvd PH 6, Key Biscayne, FL 33149...
6     (102 24th St, Miami Beach, FL 33139, USA, (25....
8     (2821 S Bayshore Dr #10b, Miami, FL 33133, USA...
10    (8701 Collins Ave, Miami Beach, FL 33154, USA,...
11    (321 Ocean Dr, Miami Beach, FL 33139, USA, (25...
12    (18501 Collins Ave #4002, Sunny Isles Beach, F...
13    (1 Collins Ave Unit 307, Miami Beach, FL 33139...
14    (17475 Collins Ave UNIT 1001, Sunny Isles Beac...
15    (Trump Palace, 18101 Collins Ave, Sunny Isles ...
16    (488 NE 18th St #4012, Miami, FL 33132, USA, (...
17    (801 Brickell Key Blvd, Miami, FL 33131, USA, ...
18    (Jade at Brickell Bay, 1331 Brickell Bay Dr #4...
19    (465 Brickell Ave APT 4301, Miami, FL 33131, U...
20    (16901 Collins Ave #5303, Sunny Isles Beach, F...
21    (360 Ocean Dr #603s, Key Biscayne, FL 33149, U...
22    (10225 Collins Ave, Bal Harbour, FL 33154,

In [20]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [21]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [22]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/3326340165.py:1: SettingWit

In [23]:
df.index

Int64Index([ 0,  1,  5,  6,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
            22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
            39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
           dtype='int64')

In [24]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [25]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,Jade Signature\r16901 Collins Ave 5305\rSunny ...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,Prime International Assoc Inc,Jade Signature 16901 Collins Ave 5305 Sunny Is...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach,"(16901 Collins Ave #5305, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
1,Bayview\r5313 Fisher Island Dr 5313\rMiami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,Luxury Living Realty,Bayview 5313 Fisher Island Dr 5313 Miami Beach,Bayview,5313 Fisher Island Dr 5313 Miami Beach,"(5313 Fisher Island Dr #5313, Miami, FL 33109,...","(25.7636181, -80.1486308, 0.0)",25.763618,-80.148631,0.0
2,Club Tower One\r789 Crandon Blvd PH-6\rKey Bis...,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,BHHS EWM Realty,Club Tower One 789 Crandon Blvd PH-6 Key Biscayne,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne,"(789 Crandon Blvd PH 6, Key Biscayne, FL 33149...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
3,1 Hotel & Homes\r102 24th St 1520\rMiami Beach,10/1/2021,6/3/2022,189,"$5,100,000.00",1386,"$3,679.65",Jennifer Restrepo,EXP Realty LLC,Jennifer Restrepo,EXP Realty LLC,1 Hotel & Homes 102 24th St 1520 Miami Beach,,1 Hotel & Homes 102 24th St 1520 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7991784, -80.1272173, 0.0)",25.799178,-80.127217,0.0
4,Two Park Grove\r2821 S Bayshore Dr 10B\rMiami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,One Sotheby's Intl Realty,Two Park Grove 2821 S Bayshore Dr 10B Miami,Two Park Grove,2821 S Bayshore Dr 10B Miami,"(2821 S Bayshore Dr #10b, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0
5,Eighty Seven Park\r8701 Collins Ave 1102/1106\...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,Douglas Elliman,Eighty Seven Park 8701 Collins Ave 1102/1106 M...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach,"(8701 Collins Ave, Miami Beach, FL 33154, USA,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
6,321 Ocean Condo\r321 Ocean Dr BH200\rMiami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,"Compass Florida, LLC",321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,"(321 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7722958, -80.1319008, 0.0)",25.772296,-80.131901,0.0
7,Turnberry Ocean Club\r18501 Collins Ave 4002\r...,3/25/2022,6/6/2022,41,"$5,500,000.00",2821,"$1,949.66",Carolina Arenas,Brown Harris Stevens,Richard Goihman,Douglas Elliman,Turnberry Ocean Club 18501 Collins Ave 4002 Su...,Turnberry Ocean Club,18501 Collins Ave 4002 Sunny Isles Beach,"(18501 Collins Ave #4002, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
8,One Ocean\r1 Collins Ave 307\rMiami Beach,9/8/2021,6/7/2022,209,"$4,550,000.00",3041,"$1,496.22",Anna Sherrill,One Sotheby's Intl Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,One Ocean 1 Collins Ave 307 Miami Beach,One Ocean,1 Collins Ave 307 Miami Beach,"(1 Collins Ave Unit 307, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
9,Chateau Beach\r17475 Collins Ave 1001\rSunny I...,7/1/2021,6/6/2022,192,"$4,500,000.00",5210,"$1,063.83",Richard Goihman,Douglas Elliman,Darin Spencer Raymond PA,BHHS EWM Realty,Chateau Beach 17475 Collins Ave 1001 Sunny Isl...,Chateau Beach,17475 Collins Ave 1001 Sunny Isles Beach,"(17475 Collins Ave 

In [27]:
df.at[3,'building_name']=('1 Hotel & Homes')
df.at[6,'building_name']=('321 Ocean')
df.at[14,'building_name']=('Icon Brickell')
df.at[24,'building_name']=('Asia')
df.at[34,'building_name']=('Biscayne Beach')
df.at[44,'building_name']=('Bella Mare')

df.at[17,'Agent']=('Bill Hernandez & Bryan Sereny')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [28]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'June 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [29]:
m.save('index.html')

In [171]:
### Clean DF for graphic ###

In [30]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [31]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/310194361.py:4: SettingWithCopyWarning: 
A value i

In [32]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [33]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

ValueError: Unable to parse string "Not Available" at position 20

In [34]:
df['year'] = 2022

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/1453407984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = 2022


In [35]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude',
       'Sale Price int', 'price_per_sqft_int', 'year'],
      dtype='object')

In [36]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
5,Eighty Seven Park\r8701 Collins Ave 1102/1106\...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach,"(8701 Collins Ave, Miami Beach, FL 33154, USA,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0,19152000.0,3540.77,2022
35,Porsche Design Tower\r18555 Collins Ave 4405\r...,2/8/2022,6/30/2022,79,"$14,400,000.00",9474,"$2,154.40",Nelson Gonzalez PA,BHHS EWM Realty,Melissa Barragan,...,Porsche Design Tower,18555 Collins Ave 4405 Sunny Isles Beach,"(18555 Collins Ave Apt 4405, Sunny Isles Beach...","(25.9481501, -80.11994659999999, 0.0)",25.948150,-80.119947,0.0,14400000.0,2154.40,2022
15,Jade Signature\r16901 Collins Ave 5303\rSunny ...,2/9/2018,6/13/2022,1566,"$10,400,000.00",10213,"$1,649.48",Rita Collins,Fortune International Realty,Fernando Alves,...,Jade Signature,16901 Collins Ave 5303 Sunny Isles Beach,"(16901 Collins Ave #5303, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,10400000.0,1649.48,2022
0,Jade Signature\r16901 Collins Ave 5305\rSunny ...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach,"(16901 Collins Ave #5305, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,10300000.0,1582.42,2022
1,Bayview\r5313 Fisher Island Dr 5313\rMiami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,...,Bayview,5313 Fisher Island Dr 5313 Miami Beach,"(5313 Fisher Island Dr #5313, Miami, FL 33109,...","(25.7636181, -80.1486308, 0.0)",25.763618,-80.148631,0.0,10000000.0,1610.57,2022
25,Oceana Key Biscayne\r360 Ocean Dr 401S\rKey Bi...,4/18/2022,6/22/2022,44,"$9,000,000.00",3791,"$2,374.04",David Siddons,Douglas Elliman,Juan De Lapuerta,...,Oceana Key Biscayne,360 Ocean Dr 401S Key Biscayne,"(360 Ocean Dr #401s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,9000000.0,2374.04,2022
6,321 Ocean Condo\r321 Ocean Dr BH200\rMiami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,...,321 Ocean,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,"(321 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7722958, -80.1319008, 0.0)",25.772296,-80.131901,0.0,9000000.0,2403.20,2022
16,Oceana Key Biscayne\r360 Ocean Dr 603S\rKey Bi...,4/11/2022,6/13/2022,7,"$7,400,000.00",3013,"$2,456.02",Giulietta Ulloa,BHHS EWM Realty,Giulietta Ulloa,...,Oceana Key Biscayne,360 Ocean Dr 603S Key Biscayne,"(360 Ocean Dr #603s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,7400000.0,2456.02,2022
2,Club Tower One\r789 Crandon Blvd PH-6\rKey Bis...,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,...,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne,"(789 Crandon Blvd PH 6, Key Biscayne, FL 33149...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0,6850000.0,2403.51,2022
36,Continuum on South Beach\r50 S Pointe Dr 2001\...,1/17/2022,6/29/2022,70,"$6,500,000.00",1900,"$3,421.05",Edward Cimino,London Foster Realty,Micky Ross-Granot,...,Continuum on South Beach,50 S Pointe Dr 2001 Miami Beach,"(50 South Pointe Dr APT 2001, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,6500000.0,3421.05,2022


In [37]:
df.sort_values('days_on_market',ascending=False)
#df.sort_values('days_on_market',ascending=True)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
20,Eighty Seven Park\r8701 Collins Ave 1205\rMiam...,Not Available,6/16/2022,Not Available,"$3,850,000.00",1522,"$2,529.57",Comp Sale,Douglas Elliman,Oliver Lloyd,...,Eighty Seven Park,8701 Collins Ave 1205 Miami Beach,"(8701 Collins Ave #1205, Miami Beach, FL 33154...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0,3850000.0,2529.57,2022
32,Paramount Worldcenter\r851 NE 1st Ave 3112\rMiami,11/19/2019,6/22/2022,900,"$1,675,000.00",2611,$721.98,Kayce Driscoll,Douglas Elliman,Sildy Cervera PA,...,Paramount Worldcenter,851 NE 1st Ave 3112 Miami,"(851 NE 1st Ave #3112, Miami, FL 33132, USA, (...","(25.782502, -80.1917783, 0.0)",25.782502,-80.191778,0.0,1675000.0,721.98,2022
4,Two Park Grove\r2821 S Bayshore Dr 10B\rMiami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,...,Two Park Grove,2821 S Bayshore Dr 10B Miami,"(2821 S Bayshore Dr #10b, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0,3999000.0,1579.38,2022
35,Porsche Design Tower\r18555 Collins Ave 4405\r...,2/8/2022,6/30/2022,79,"$14,400,000.00",9474,"$2,154.40",Nelson Gonzalez PA,BHHS EWM Realty,Melissa Barragan,...,Porsche Design Tower,18555 Collins Ave 4405 Sunny Isles Beach,"(18555 Collins Ave Apt 4405, Sunny Isles Beach...","(25.9481501, -80.11994659999999, 0.0)",25.948150,-80.119947,0.0,14400000.0,2154.40,2022
22,Turnberry Ocean Colony\r16047 Collins Ave 702\...,1/24/2022,6/15/2022,78,"$2,384,000.00",2680,"$1,061.92",Lana Bell,Douglas Elliman,Dominique Gagnon,...,Turnberry Ocean Colony,16047 Collins Ave 702 Sunny Isles Beach,"(16047 Collins Ave APT 702, Sunny Isles Beach,...","(25.9252659, -80.1216882, 0.0)",25.925266,-80.121688,0.0,2384000.0,1061.92,2022
27,Muse\r17141 Collins Ave 1201\rSunny Isles Beach,3/6/2022,6/21/2022,75,"$4,750,000.00",3195,"$1,486.70",Sandra Pariente,Charles Rutenberg Realty,Neal Klotsman,...,Muse,17141 Collins Ave 1201 Sunny Isles Beach,"(17141 Collins Ave #1201, Sunny Isles Beach, F...","(25.934944, -80.12091819999999, 0.0)",25.934944,-80.120918,0.0,4750000.0,1486.70,2022
36,Continuum on South Beach\r50 S Pointe Dr 2001\...,1/17/2022,6/29/2022,70,"$6,500,000.00",1900,"$3,421.05",Edward Cimino,London Foster Realty,Micky Ross-Granot,...,Continuum on South Beach,50 S Pointe Dr 2001 Miami Beach,"(50 South Pointe Dr APT 2001, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,6500000.0,3421.05,2022
16,Oceana Key Biscayne\r360 Ocean Dr 603S\rKey Bi...,4/11/2022,6/13/2022,7,"$7,400,000.00",3013,"$2,456.02",Giulietta Ulloa,BHHS EWM Realty,Giulietta Ulloa,...,Oceana Key Biscayne,360 Ocean Dr 603S Key Biscayne,"(360 Ocean Dr #603s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,7400000.0,2456.02,2022
38,Murano at Portofino Condo\r1000 S Pointe Dr 35...,3/30/2022,6/27/2022,69,"$5,125,000.00",2618,"$1,957.60",Dina Goldentayer,Douglas Elliman,Julian Cohen,...,Murano at Portofino Condo,1000 S Pointe Dr 3502 Miami Beach,"(1000 South Pointe Dr APT 3502, Miami Beach, F...","(25.7690044, -80.1381107, 0.0)",25.769004,-80.138111,0.0,5125000.0,1957.60,2022
28,Spear at Aqua Condo\r6103 Aqua Ave 1004\rMiami...,2/3/2022,6/21/2022,68,"$2,400,000.00",2203,"$1,089.42",Jeffrey Miller,One Sotheby's International,Jason Angrist,...,Spear at Aqua Condo,6103 Aqua Ave 1004 Miami Beach,"(6103 Aqua Ave APT 1004, Miami Beach, FL 33141...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0,2400000.0,1089.42,2022


### Sale price sum

In [38]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$219,075,000.00'

### Sale price average

In [39]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$4,868,333.33'

### Price per sf average

In [40]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,683.98'

### Price per square fot min vs. max

In [41]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$6,605.02'

In [42]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$558.09'

### Sale price min and max

In [43]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$19,152,000.00'

In [44]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$1,500,000.00'

In [45]:
len(df)

45

In [46]:
df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_77453/584204004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))


,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Jade Signature 16901 Collins Ave 5305 Sunny Is...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach,"(16901 Collins Ave #5305, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,10300000.0,1582.42,2022
1,Bayview 5313 Fisher Island Dr 5313 Miami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,...,Bayview,5313 Fisher Island Dr 5313 Miami Beach,"(5313 Fisher Island Dr #5313, Miami, FL 33109,...","(25.7636181, -80.1486308, 0.0)",25.763618,-80.148631,0.0,10000000.0,1610.57,2022
2,Club Tower One 789 Crandon Blvd PH-6 Key Biscayne,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,...,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne,"(789 Crandon Blvd PH 6, Key Biscayne, FL 33149...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0,6850000.0,2403.51,2022
3,1 Hotel & Homes 102 24th St 1520 Miami Beach,10/1/2021,6/3/2022,189,"$5,100,000.00",1386,"$3,679.65",Jennifer Restrepo,EXP Realty LLC,Jennifer Restrepo,...,1 Hotel & Homes,1 Hotel & Homes 102 24th St 1520 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7991784, -80.1272173, 0.0)",25.799178,-80.127217,0.0,5100000.0,3679.65,2022
4,Two Park Grove 2821 S Bayshore Dr 10B Miami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,...,Two Park Grove,2821 S Bayshore Dr 10B Miami,"(2821 S Bayshore Dr #10b, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0,3999000.0,1579.38,2022
5,Eighty Seven Park 8701 Collins Ave 1102/1106 M...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach,"(8701 Collins Ave, Miami Beach, FL 33154, USA,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0,19152000.0,3540.77,2022
6,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,...,321 Ocean,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,"(321 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7722958, -80.1319008, 0.0)",25.772296,-80.131901,0.0,9000000.0,2403.20,2022
7,Turnberry Ocean Club 18501 Collins Ave 4002 Su...,3/25/2022,6/6/2022,41,"$5,500,000.00",2821,"$1,949.66",Carolina Arenas,Brown Harris Stevens,Richard Goihman,...,Turnberry Ocean Club,18501 Collins Ave 4002 Sunny Isles Beach,"(18501 Collins Ave #4002, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0,5500000.0,1949.66,2022
8,One Ocean 1 Collins Ave 307 Miami Beach,9/8/2021,6/7/2022,209,"$4,550,000.00",3041,"$1,496.22",Anna Sherrill,One Sotheby's Intl Realty,Ana Teresa Rodriguez,...,One Ocean,1 Collins Ave 307 Miami Beach,"(1 Collins Ave Unit 307, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0,4550000.0,1496.22,2022
9,Chateau Beach 17475 Collins Ave 1001 Sunny Isl...,7/1/2021,6/6/2022,192,"$4,500,000.00",5210,"$1,063.83",Richard Goihman,Douglas Elliman,Darin Spencer Raymond PA,...,Chateau Beach,17475 Collins Ave 1001 Sunny Isles Beach,"(17475 Collins Ave UNIT 1001, Sunny Isles Beac...","(25.9379684, -80.1207198, 0.0)",25.937968,-80.120720,0.0,4500000.0,1063.83,2022


In [47]:
df['year'] = '2021'

In [48]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Jade Signature 16901 Collins Ave 5305 Sunny Is...,4/29/2019,6/1/2022,1060,"$10,300,000.00",7916,"$1,582.42",Sandra Chartouni,Fortune International Realty,Daniel Alitenssi,...,Jade Signature,16901 Collins Ave 5305 Sunny Isles Beach,"(16901 Collins Ave #5305, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0,10300000.0,1582.42,2021
1,Bayview 5313 Fisher Island Dr 5313 Miami Beach,12/22/2021,6/1/2022,65,"$10,000,000.00",6209,"$1,610.57",Karla Abaunza,Luxury Living Realty,Karla Abaunza,...,Bayview,5313 Fisher Island Dr 5313 Miami Beach,"(5313 Fisher Island Dr #5313, Miami, FL 33109,...","(25.7636181, -80.1486308, 0.0)",25.763618,-80.148631,0.0,10000000.0,1610.57,2021
2,Club Tower One 789 Crandon Blvd PH-6 Key Biscayne,2/24/2022,6/1/2022,11,"$6,850,000.00",2850,"$2,403.51",Claudine Coto,One Sotheby's Intl Realty,Darin Spencer Raymond PA,...,Club Tower One,789 Crandon Blvd PH-6 Key Biscayne,"(789 Crandon Blvd PH 6, Key Biscayne, FL 33149...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0,6850000.0,2403.51,2021
3,1 Hotel & Homes 102 24th St 1520 Miami Beach,10/1/2021,6/3/2022,189,"$5,100,000.00",1386,"$3,679.65",Jennifer Restrepo,EXP Realty LLC,Jennifer Restrepo,...,1 Hotel & Homes,1 Hotel & Homes 102 24th St 1520 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7991784, -80.1272173, 0.0)",25.799178,-80.127217,0.0,5100000.0,3679.65,2021
4,Two Park Grove 2821 S Bayshore Dr 10B Miami,2/7/2022,6/1/2022,8,"$3,999,000.00",2532,"$1,579.38",Pietro Belmonte,Douglas Elliman,Rose Harris,...,Two Park Grove,2821 S Bayshore Dr 10B Miami,"(2821 S Bayshore Dr #10b, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0,3999000.0,1579.38,2021
5,Eighty Seven Park 8701 Collins Ave 1102/1106 M...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach,"(8701 Collins Ave, Miami Beach, FL 33154, USA,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0,19152000.0,3540.77,2021
6,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,...,321 Ocean,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,"(321 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7722958, -80.1319008, 0.0)",25.772296,-80.131901,0.0,9000000.0,2403.20,2021
7,Turnberry Ocean Club 18501 Collins Ave 4002 Su...,3/25/2022,6/6/2022,41,"$5,500,000.00",2821,"$1,949.66",Carolina Arenas,Brown Harris Stevens,Richard Goihman,...,Turnberry Ocean Club,18501 Collins Ave 4002 Sunny Isles Beach,"(18501 Collins Ave #4002, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0,5500000.0,1949.66,2021
8,One Ocean 1 Collins Ave 307 Miami Beach,9/8/2021,6/7/2022,209,"$4,550,000.00",3041,"$1,496.22",Anna Sherrill,One Sotheby's Intl Realty,Ana Teresa Rodriguez,...,One Ocean,1 Collins Ave 307 Miami Beach,"(1 Collins Ave Unit 307, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0,4550000.0,1496.22,2021
9,Chateau Beach 17475 Collins Ave 1001 Sunny Isl...,7/1/2021,6/6/2022,192,"$4,500,000.00",5210,"$1,063.83",Richard Goihman,Douglas Elliman,Darin Spencer Raymond PA,...,Chateau Beach,17475 Collins Ave 1001 Sunny Isles Beach,"(17475 Collins Ave UNIT 1001, Sunny Isles Beac...","(25.9379684, -80.1207198, 0.0)",25.937968,-80.120720,0.0,4500000.0,1063.83,2021


In [49]:
df.to_csv('June2022CondoSales.csv', sep='|')

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [51]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [52]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesJune2022
